# ESTANDARIZACIÓN DE COLONIAS

In [1]:
import pandas as pd # Manipulación df
import os # Directorios
import re # Manipulación de cadenas de texto (expresiones regulares)
import unicodedata # Función para eliminar acentos

In [2]:
#df = pd.read_csv("C:/xampp/htdocs/datalpine/resources/db/Scrapining/ciudades/puebla/clean/agosto_2024.csv")
df = pd.read_csv("C:/xampp/htdocs/datalpine/resources/db/Scrapining/ciudades/tulancingo/clean/jul_2024.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         141 non-null    int64  
 1   categoria                  134 non-null    object 
 2   precio                     141 non-null    int64  
 3   precio_mxn                 141 non-null    int64  
 4   precio_usd                 141 non-null    float64
 5   fecha_conversion           141 non-null    object 
 6   propiedad                  141 non-null    object 
 7   metros_total               141 non-null    int64  
 8   metros_construido          141 non-null    int64  
 9   precio_m2_terreno          141 non-null    float64
 10  precio_m2_construido       141 non-null    float64
 11  tiempo_de_publicacion      141 non-null    object 
 12  meses_transcurridos        141 non-null    int64  
 13  meses_transcurridos_fecha  141 non-null    object 

# Separación

In [3]:
# Siempre regresar una lista de 3 elementos
def split_location(location):
    parts = location.split(',')
    parts += [None] * (3 - len(parts))
    return parts[:3] 
df[['Colonia', 'Municipio', 'Estado']] = df['ubicacion'].apply(split_location).apply(pd.Series)

# Limpiando valores nulos
df['Colonia'] = df['Colonia'].str.strip().fillna('Sin_colonia')
df['Municipio'] = df['Municipio'].str.strip().fillna('Sin_municipio')
df['Estado'] = df['Estado'].str.strip().fillna('Sin_estado')
df[['ubicacion', 'Colonia', 'Municipio', 'Estado']]

,ubicacion,Colonia,Municipio,Estado
0,"20 de Noviembre, Tulancingo de Bravo, Hidalgo",20 de Noviembre,Tulancingo de Bravo,Hidalgo
1,"20 de Noviembre, Tulancingo de Bravo, Hidalgo",20 de Noviembre,Tulancingo de Bravo,Hidalgo
2,"20 de Noviembre, Tulancingo de Bravo, Hidalgo",20 de Noviembre,Tulancingo de Bravo,Hidalgo
3,"20 de Noviembre, Tulancingo de Bravo, Hidalgo",20 de Noviembre,Tulancingo de Bravo,Hidalgo
4,"20 de Noviembre, Tulancingo de Bravo, Hidalgo",20 de Noviembre,Tulancingo de Bravo,Hidalgo
...,...,...,...,...
136,"Insurgentes, Tulancingo de Bravo, Hidalgo",Insurgentes,Tulancingo de Bravo,Hidalgo
137,"Tulancingo Centro, Tulancingo de Bravo, Hidalgo",Tulancingo Centro,Tulancingo de Bravo,Hidalgo
138,"Jardines Del Sur, Tulancingo de Bravo, Hidalgo",Jardines Del Sur,Tulancingo de Bravo,Hidalgo
139,"Felipe Angeles, Tulancingo de Bravo, Hidalgo",Felipe Angeles,Tulancingo de Bravo,Hidalgo


In [4]:
# Aplicando valor de acuerdo a la base a limpiar
df['Estado'] = df['Estado'].replace('Sin_estado', 'Hidalgo')

### Estandarizar

In [5]:
# Minúsculas
df['Colonia'] = df['Colonia'].str.strip().str.lower()
df['Municipio'] = df['Municipio'].str.strip().str.lower()
df['Estado'] = df['Estado'].str.strip().str.lower()

In [6]:
# Esta línea de código aplica la función unicodedata.normalize a la columna 'Colonia' del DataFrame df.
# El argumento 'NFKD' indica que se debe aplicar la normalización NFKD, que elimina los acentos y otros caracteres diacríticos.
#df['Colonia'] = df['Colonia'].apply(lambda x: unicodedata.normalize('NFKD', x))

import unicodedata# Función para eliminar acentos
def eliminar_acentos(texto):
    # Normaliza y elimina los acentos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    return ''.join(char for char in texto_normalizado if unicodedata.category(char) != 'Mn')
df['Colonia'] = df['Colonia'].apply(eliminar_acentos)
df['Municipio'] = df['Municipio'].apply(eliminar_acentos)
df['Estado'] = df['Estado'].apply(eliminar_acentos)
print(df['Colonia'].unique())

['20 de noviembre' 'jardines del sur' 'vicente guerrero' 'jaltepec'
 'lomas de progreso' 'francisco i madero' 'la morena' 'huapalcalco'
 'cebolletas' 'los pinos' 'pedregal de san francisco' 'napateco'
 'tulancingo centro' 'san juan' 'el mirador' 'insurgentes' 'rojo gomez'
 'san jose caltengo' 'medias tierras' 'lindavista' 'caltengo'
 'valle verde' 'la herradura' 'el banco' 'benito juarez' 'santa ana'
 'felipe angeles']


### Separación de tipo de lugar, colonias

In [7]:
# Añade las diferentes formas de identificar el tipo de lugar (etiquetas de ubicación)
tipos = ['condominio', 'fraccionamiento', 'infonavit','colonia', 'frac','fracc','fraccio','fracc.','namiento','ionamiento', 'uhp', 'urbanización','urbanizacion', 'sector', 'barrio', 'zona', 'sector', 'unidad habitacional', ]
#, 'residencial','unidad residencial'
def obtener_tipo_lugar(nombre):
    for tipo in tipos:
        if tipo in nombre:
            return tipo
    return None

df['tipo_lugar'] = df['Colonia'].apply(obtener_tipo_lugar)
# Eliminar el tipo de lugar de la columna 'colonia'
df['Colonia'] = df.apply(lambda row: row['Colonia'].replace(row['tipo_lugar'], '').strip() if row['tipo_lugar'] else row['Colonia'], axis=1)
print(df[['Colonia','tipo_lugar','ubicacion']])
print(df['tipo_lugar'].unique())

               Colonia tipo_lugar  \
0      20 de noviembre       None   
1      20 de noviembre       None   
2      20 de noviembre       None   
3      20 de noviembre       None   
4      20 de noviembre       None   
..                 ...        ...   
136        insurgentes       None   
137  tulancingo centro       None   
138   jardines del sur       None   
139     felipe angeles       None   
140  tulancingo centro       None   

                                           ubicacion  
0      20 de Noviembre, Tulancingo de Bravo, Hidalgo  
1      20 de Noviembre, Tulancingo de Bravo, Hidalgo  
2      20 de Noviembre, Tulancingo de Bravo, Hidalgo  
3      20 de Noviembre, Tulancingo de Bravo, Hidalgo  
4      20 de Noviembre, Tulancingo de Bravo, Hidalgo  
..                                               ...  
136        Insurgentes, Tulancingo de Bravo, Hidalgo  
137  Tulancingo Centro, Tulancingo de Bravo, Hidalgo  
138   Jardines Del Sur, Tulancingo de Bravo, Hidalgo  
139  

### Mapeo con diccionario estandarizado

In [8]:
diccionario = pd.read_csv('colonias_estandarizadas.csv')
diccionario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3143 non-null   int64  
 1   categoria                  2930 non-null   object 
 2   precio                     3143 non-null   float64
 3   precio_mxn                 3143 non-null   float64
 4   precio_usd                 3143 non-null   int64  
 5   fecha_conversion           3143 non-null   object 
 6   propiedad                  3143 non-null   object 
 7   metros_total               3143 non-null   int64  
 8   metros_construido          3143 non-null   int64  
 9   precio_m2_terreno          3143 non-null   float64
 10  precio_m2_construido       3143 non-null   float64
 11  tiempo_de_publicacion      3143 non-null   object 
 12  meses_transcurridos        3143 non-null   int64  
 13  meses_transcurridos_fecha  3143 non-null   objec

In [9]:
#  Función para estandarizar colonias
def estand_col(colonia, diccionario):
    return diccionario[diccionario['Colonia'] == colonia]['Colonia_Estandarizada'].values[0] if colonia in diccionario['Colonia'].values else colonia
df['Colonia_Estandarizada'] = df['Colonia'].apply(lambda x: estand_col(x, diccionario))
df[['Colonia_Estandarizada','Colonia']]

,Colonia_Estandarizada,Colonia
0,20 de noviembre,20 de noviembre
1,20 de noviembre,20 de noviembre
2,20 de noviembre,20 de noviembre
3,20 de noviembre,20 de noviembre
4,20 de noviembre,20 de noviembre
...,...,...
136,insurgentes,insurgentes
137,tulancingo centro,tulancingo centro
138,jardines del sur,jardines del sur
139,felipe angeles,felipe angeles


In [15]:
# Función para obtener el código postal dado estado, municipio, y colonia
def obtener_codigo_postal(colonia, municipio, estado, df_codigos_postales):
    # Normalizar los inputs
    estado = estado.lower().strip()
    municipio = municipio.lower().strip()
    colonia = colonia.lower().strip()
    
    estado = estado.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))
    municipio = municipio.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))
    colonia = colonia.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))

    # Filtrar el DataFrame según estado y municipio
    df_filtrado = df_codigos_postales[(df_codigos_postales['estado'] == estado) & (df_codigos_postales['D_mnpio'] == municipio)]
    
    # Intentar encontrar coincidencias exactas de la colonia
    resultado = df_filtrado[df_filtrado['d_asenta'] == colonia]
    
    if not resultado.empty:
        # Si se encuentra una coincidencia, devolver el código postal
        return resultado['d_codigo'].values[0]
    
    return None  # Devolver None si no hay coincidencia

In [16]:
df['codigo_postal'] = df.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)

### Corregir por similitud

In [17]:
archivo_excel = 'D:/yoe11/Documents/TESEO/Modelo Factible/Scraplining/CPdescarga.xlsx'
hojas_excel = pd.ExcelFile(archivo_excel, engine='openpyxl')
# Imprimir los nombres de las hojas
print(hojas_excel.sheet_names)

['Nota', 'Aguascalientes', 'Baja_California', 'Baja_California_Sur', 'Campeche', 'Coahuila_de_Zaragoza', 'Colima', 'Chiapas', 'Chihuahua', 'Distrito_Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'México', 'Michoacán_de_Ocampo', 'Morelos', 'Nayarit', 'Nuevo_León', 'Querétaro', 'Oaxaca', 'Puebla', 'Quintana_Roo', 'Sonora', 'San_Luis_Potosí', 'Sinaloa', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz_de_Ignacio_de_la_Llave', 'Yucatán', 'Zacatecas']


In [18]:
# Obtener el DataFrame específico para el estado 'Puebla'
df_codigos_postales = pd.read_excel(archivo_excel, sheet_name='Hidalgo', engine='openpyxl', usecols=['D_mnpio', 'd_asenta', 'd_tipo_asenta', 'd_codigo'])
df_codigos_postales['estado'] = 'Hidalgo'  # Agregar el nombre del estado como 'estado'
df_codigos_postales.head()

,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,estado
0,42000,Centro,Colonia,Pachuca de Soto,Hidalgo
1,42004,La Granada,Colonia,Pachuca de Soto,Hidalgo
2,42010,La Alcantarilla,Barrio,Pachuca de Soto,Hidalgo
3,42010,El Arbolito,Barrio,Pachuca de Soto,Hidalgo
4,42010,El Atorón,Barrio,Pachuca de Soto,Hidalgo


In [19]:
df_codigos_postales = df_codigos_postales[df_codigos_postales['D_mnpio'].str.contains('Tulancingo de Bravo')]
df_codigos_postales['d_asenta'].unique()

array(['Tulancingo Centro', 'Del Villar', 'Nueva Morelos',
       'De los Electricistas', 'Los Pinos', 'Real de Minas',
       'Nuevo Tulancingo', 'Hidalgo Unido', 'Lomas de Progreso',
       'La Cañada', 'La Central', 'Villas de la Cañada',
       'San Nicolás el Chico', 'Unidad Deportiva',
       'Rincones de la Hacienda', 'Residencial Santa Fe',
       'Privada Riva Palacio', 'Minera Secc 200', 'Los Álamos INFONAVIT',
       'Privada los Mesones', 'San Daniel', 'Alamoxtitla',
       'Rinconada la Morena', 'La Morena Sección Norte "C"',
       'La Morena 3a. Sección', 'Manuel M. Ponce',
       'Boulevares de Tulancingo', 'La Morena Sección Norte A',
       'La Morena Sección Norte "B"', 'Caltengo', 'San José Caltengo',
       'Santa Clemencia', 'La Argentina', 'El Refugio', 'El Volcán',
       'Polanco', 'Jardines de Napateco', 'Huapalcalco',
       'Parque Urbano Napateco', 'Napateco', 'Los Sabinos',
       'La Loma Napateco', 'Sultepec', 'Francisco Villa Napateco',
       'Liz Napa

In [24]:
df_codigos_postales['d_asenta'].unique()

array(['tulancingo centro', 'del villar', 'nueva morelos',
       'de los electricistas', 'los pinos', 'real de minas',
       'nuevo tulancingo', 'hidalgo unido', 'lomas de progreso',
       'la canada', 'la central', 'villas de la canada',
       'san nicolas el chico', 'unidad deportiva',
       'rincones de la hacienda', 'residencial santa fe',
       'privada riva palacio', 'minera secc 200', 'los alamos infonavit',
       'privada los mesones', 'san daniel', 'alamoxtitla',
       'rinconada la morena', 'la morena seccion norte "c"',
       'la morena 3a. seccion', 'manuel m. ponce',
       'boulevares de tulancingo', 'la morena seccion norte a',
       'la morena seccion norte "b"', 'caltengo', 'san jose caltengo',
       'santa clemencia', 'la argentina', 'el refugio', 'el volcan',
       'polanco', 'jardines de napateco', 'huapalcalco',
       'parque urbano napateco', 'napateco', 'los sabinos',
       'la loma napateco', 'sultepec', 'francisco villa napateco',
       'liz napa

In [25]:
# Limpiar y estandarizar las columnas
df_codigos_postales['D_mnpio'] = df_codigos_postales['D_mnpio'].str.lower().str.strip()
df_codigos_postales['d_asenta'] = df_codigos_postales['d_asenta'].str.lower().str.strip()
df_codigos_postales['d_tipo_asenta'] = df_codigos_postales['d_tipo_asenta'].str.lower().str.strip()
df_codigos_postales['estado'] = df_codigos_postales['estado'].str.lower().str.strip()

In [26]:
# Eliminar acentos de todas las columnas de df_codigos_postales para que se pueda mapear bien
df_codigos_postales['d_asenta'] = df_codigos_postales['d_asenta'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', errors='ignore').decode('utf-8') if isinstance(x, str) else x)

In [14]:
# Mapear colonias de df_codigos_postales a las colonias de df para estandarizar

# Coincidencias exactas
#df['Colonia_Estandarizada'] = df['Colonia'].apply(lambda x: df_codigos_postales[df_codigos_postales['d_asenta'].str.lower() == x.lower()]['d_asenta'].values[0] if not df_codigos_postales[df_codigos_postales['d_asenta'].str.lower() == x.lower()].empty else x)
# Coincidencias parciales
#df['Colonia_Estandarizada'] = df['Colonia'].apply(lambda x: df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(x, case=False)]['d_asenta'].values[0] if not df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(x, case=False)].empty else x)

#print(df[['tipo_lugar','Colonia','Colonia_Estandarizada','Municipio','Estado','ubicacion']])

### Primera estandarización, Por mapeo en coincidencia flexible (fuzzy)

In [27]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

# Función para mapear colonias con coincidencia flexible
def mapear_colonia_flexible(colonia, df_codigos_postales):
    # Lista de todas las colonias en el diccionario
    colonias_diccionario = df_codigos_postales['d_asenta'].unique()

    # Buscar la coincidencia más cercana con fuzzywuzzy
    mejor_coincidencia = process.extractOne(colonia, colonias_diccionario, scorer=fuzz.ratio, score_cutoff=80)  # Ajusta el threshold de similitud a 80
    
    # Si se encuentra una coincidencia, devolverla; de lo contrario, devolver el valor original
    return mejor_coincidencia[0] if mejor_coincidencia else colonia

# Aplicar la función al DataFrame original
df['Colonia_Estandarizada'] = df['Colonia'].apply(lambda x: mapear_colonia_flexible(x, df_codigos_postales))
print(df[['tipo_lugar', 'Colonia', 'Colonia_Estandarizada', 'Municipio', 'Estado', 'ubicacion']])

    tipo_lugar            Colonia Colonia_Estandarizada            Municipio  \
0         None    20 de noviembre       20 de noviembre  tulancingo de bravo   
1         None    20 de noviembre       20 de noviembre  tulancingo de bravo   
2         None    20 de noviembre       20 de noviembre  tulancingo de bravo   
3         None    20 de noviembre       20 de noviembre  tulancingo de bravo   
4         None    20 de noviembre       20 de noviembre  tulancingo de bravo   
..         ...                ...                   ...                  ...   
136       None        insurgentes           insurgentes  tulancingo de bravo   
137       None  tulancingo centro     tulancingo centro  tulancingo de bravo   
138       None   jardines del sur      jardines del sur  tulancingo de bravo   
139       None     felipe angeles        felipe angeles  tulancingo de bravo   
140       None  tulancingo centro     tulancingo centro  tulancingo de bravo   

      Estado                           

In [28]:
df.shape

(141, 32)

In [16]:
#from fuzzywuzzy import fuzz, process

#def tokenizar_y_mapear(colonia, df_codigos_postales):
    # Tokenizar la colonia (eliminar duplicados y ordenar tokens)
#    tokens_colonia = ' '.join(sorted(set(colonia.lower().split())))

    # Lista de todas las colonias en el diccionario, también tokenizadas
#    colonias_diccionario = df_codigos_postales['d_asenta'].apply(lambda x: ' '.join(sorted(set(x.lower().split()))))

    # Usar fuzzywuzzy para encontrar la mejor coincidencia tokenizada
#    mejor_coincidencia = process.extractOne(tokens_colonia, colonias_diccionario, scorer=fuzz.token_sort_ratio, score_cutoff=80)
    
    # Devolver la mejor coincidencia o el valor original si no hay coincidencia
#    return mejor_coincidencia[0] if mejor_coincidencia else colonia

# Aplicar la función al DataFrame original
#df['Colonia_Estandarizada'] = df['Colonia'].apply(lambda x: tokenizar_y_mapear(x, df_codigos_postales))

# Mostrar los resultados
#print(df[['tipo_lugar', 'Colonia', 'Colonia_Estandarizada', 'Municipio', 'Estado', 'ubicacion']])

In [29]:
# Función para obtener el código postal dado estado, municipio, y colonia
def obtener_codigo_postal(colonia, municipio, estado, df_codigos_postales):
    # Normalizar los inputs
    estado = estado.lower().strip()
    municipio = municipio.lower().strip()
    colonia = colonia.lower().strip()
    
    estado = estado.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))
    municipio = municipio.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))
    colonia = colonia.lower().strip().translate(str.maketrans('', '', 'áéíóúÁÉÍÓÚ'))

    # Filtrar el DataFrame según estado y municipio
    df_filtrado = df_codigos_postales[(df_codigos_postales['estado'] == estado) & (df_codigos_postales['D_mnpio'] == municipio)]
    
    # Intentar encontrar coincidencias exactas de la colonia
    resultado = df_filtrado[df_filtrado['d_asenta'] == colonia]
    
    if not resultado.empty:
        # Si se encuentra una coincidencia, devolver el código postal
        return resultado['d_codigo'].values[0]
    
    return None  # Devolver None si no hay coincidencia

In [30]:
df['codigo_postal'] = df.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
# Verificar si hay registros sin código postal asignado

sin_codigo_postal = df[df['codigo_postal'].isna()]
Limpios = df[df['codigo_postal'].notna()]
print(sin_codigo_postal[['Colonia','Municipio','Estado','codigo_postal']])
print(sin_codigo_postal.shape)

        Colonia            Municipio   Estado  codigo_postal
26    la morena  tulancingo de bravo  hidalgo            NaN
27    la morena  tulancingo de bravo  hidalgo            NaN
28    la morena  tulancingo de bravo  hidalgo            NaN
29    la morena  tulancingo de bravo  hidalgo            NaN
32    la morena  tulancingo de bravo  hidalgo            NaN
33    la morena  tulancingo de bravo  hidalgo            NaN
34    la morena  tulancingo de bravo  hidalgo            NaN
39    la morena  tulancingo de bravo  hidalgo            NaN
40    la morena  tulancingo de bravo  hidalgo            NaN
41    la morena  tulancingo de bravo  hidalgo            NaN
42    la morena  tulancingo de bravo  hidalgo            NaN
43    la morena  tulancingo de bravo  hidalgo            NaN
44    la morena  tulancingo de bravo  hidalgo            NaN
45    la morena  tulancingo de bravo  hidalgo            NaN
46    la morena  tulancingo de bravo  hidalgo            NaN
48    la morena  tulanci

###  Segunda estandarización, por coincidencias parciales

In [ ]:
df['Colonia'] = df['Colonia'].apply(lambda x: 'centro' if x == 'centro' else x)
#df['Colonia'] = df['Colonia'].apply(lambda x: 'plazas amalucan' if x == 'plazas de amalucan' else x)
df['Colonia'] = df['Colonia'].apply(lambda x: 'centro' if x == 'puebla' else x)
df['Colonia'] = df['Colonia'].apply(lambda x: 'centro' if x == 'puebla centro' else x)

In [19]:
# Mapear colonias de df_codigos_postales a las colonias de df para estandarizar

# Coincidencias exactas
#sin_codigo_postal['Colonia_Estandarizada'] = sin_codigo_postal['Colonia'].apply(lambda x: df_codigos_postales[df_codigos_postales['d_asenta'].str.lower() == x.lower()]['d_asenta'].values[0] if not df_codigos_postales[df_codigos_postales['d_asenta'].str.lower() == x.lower()].empty else x)
# Coincidencias parciales
sin_codigo_postal.loc[:, 'Colonia_Estandarizada'] = sin_codigo_postal['Colonia'].apply(lambda x: df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(x, case=False)]['d_asenta'].values[0] if not df_codigos_postales[df_codigos_postales['d_asenta'].str.contains(x, case=False)].empty else x)
print(sin_codigo_postal[['Colonia','Colonia_Estandarizada','Municipio','Estado','ubicacion']].head(40))

                         Colonia           Colonia_Estandarizada Municipio  \
0                  puebla centro                   puebla centro    puebla   
7                  puebla centro                   puebla centro    puebla   
17                       la loma    la loma (ejido romero vargas    puebla   
34          san miguel mayorazgo  infonavit san miguel mayorazgo    puebla   
35          san miguel mayorazgo  infonavit san miguel mayorazgo    puebla   
86          san miguel mayorazgo  infonavit san miguel mayorazgo    puebla   
97                   san bartolo           infonavit san bartolo    puebla   
115                puebla centro                   puebla centro    puebla   
119                   los heroes         los heroes de puebla ii    puebla   
120                   los heroes         los heroes de puebla ii    puebla   
121         san miguel mayorazgo  infonavit san miguel mayorazgo    puebla   
127                   3 cerritos                      3 cerritos

In [20]:
sin_codigo_postal.loc[:, 'codigo_postal'] = sin_codigo_postal.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
# Verificar si hay registros sin código postal asignado
sin_codigo_postal_2 = sin_codigo_postal[sin_codigo_postal['codigo_postal'].isna()]
Limpios2 = sin_codigo_postal[sin_codigo_postal['codigo_postal'].notna()]

print(sin_codigo_postal_2[['Colonia','Municipio','Estado','codigo_postal']])
print(sin_codigo_postal_2.shape)

               Colonia Municipio  Estado  codigo_postal
0        puebla centro    puebla  puebla            NaN
7        puebla centro    puebla  puebla            NaN
115      puebla centro    puebla  puebla            NaN
127         3 cerritos    puebla  puebla            NaN
130      puebla centro    puebla  puebla            NaN
...                ...       ...     ...            ...
3464  centro historico    puebla  puebla            NaN
3465  centro historico    puebla  puebla            NaN
3466  centro historico    puebla  puebla            NaN
3470  centro historico    puebla  puebla            NaN
3475  centro historico    puebla  puebla            NaN

[349 rows x 4 columns]
(349, 32)


In [21]:
sin_codigo_postal_2['Colonia'].unique()

array(['puebla centro', '3 cerritos', 'residencial villa encantada',
       'centro historico', 'hacienda del camino real', 'snte',
       'villas de atlixco', 'bellavista', 'las 3 cruces',
       'hacienda zavaleta', 'villa olimpica', '2 de abril',
       'san vicente ferrer', 'lomas de angelopolis',
       'fuentes de angelopolis', 'residencial puebla', '6 de enero',
       'agricola villa albertina', '3 de mayo', 'las margaritas',
       'residencial zavaleta', '10 de mayo', 'rincon de san andres',
       'bugambilias 3a secc', 'el barreal', 'geo villas atlixcayotl',
       'momosa', 'pueblo nuevo', 'gral. ignacio zaragoza',
       'villas san carlos', 'camino real a cholula', '20 de noviembre',
       'momoxpan ii', 'concepcion la cruz', 'concepcion guadalupe',
       'residencial del encanto', 'francisco i madero', 'villa la noria',
       'villas zavaleta primera seccion', 'santa gema',
       'villa de zavaleta', 'altavista', 'la concepcion',
       'las fuentes de puebla', 'ins

### Convirtiendo números a palabras y coincidencias parciales segunda vez

In [22]:
# Diccionario para convertir números cardinales a palabras
numeros_cardinales = {
    '1': 'uno',
    '2': 'dos',
    '3': 'tres',
    '4': 'cuatro',
    '5': 'cinco',
    '6': 'seis',
    '7': 'siete',
    '8': 'ocho',
    '9': 'nueve',
    '10': 'diez',
}

# Diccionario para convertir números ordinales a palabras
numeros_ordinales = {
    '1ra': 'primera',
    '2da': 'segunda',
    '3ra': 'tercera',
    '4ta': 'cuarta',
    '5ta': 'quinta',
    '6ta': 'sexta',
    '7ma': 'séptima',
    '8va': 'octava',
    '9na': 'novena',
    '10ma': 'décima',
    # Agregar más ordinales si es necesario
}
import re

def reemplazar_numeros(colonia):
    # Primero reemplazar los ordinales
    for num_ord, palabra_ord in numeros_ordinales.items():
        colonia = re.sub(rf'\b{num_ord}\b', palabra_ord, colonia)
    
    # Luego reemplazar los números cardinales aislados
    for num_card, palabra_card in numeros_cardinales.items():
        colonia = re.sub(rf'\b{num_card}\b', palabra_card, colonia)
    
    return colonia

# Utilizar .loc para asignar el resultado de la aplicación de la función reemplazar_numeros
sin_codigo_postal_2.loc[:, 'Colonia_Estandarizada'] = sin_codigo_postal_2['Colonia'].apply(reemplazar_numeros)
sin_codigo_postal_2['Colonia_Estandarizada'].unique()

array(['puebla centro', 'tres cerritos', 'residencial villa encantada',
       'centro historico', 'hacienda del camino real', 'snte',
       'villas de atlixco', 'bellavista', 'las tres cruces',
       'hacienda zavaleta', 'villa olimpica', 'dos de abril',
       'san vicente ferrer', 'lomas de angelopolis',
       'fuentes de angelopolis', 'residencial puebla', 'seis de enero',
       'agricola villa albertina', 'tres de mayo', 'las margaritas',
       'residencial zavaleta', 'diez de mayo', 'rincon de san andres',
       'bugambilias 3a secc', 'el barreal', 'geo villas atlixcayotl',
       'momosa', 'pueblo nuevo', 'gral. ignacio zaragoza',
       'villas san carlos', 'camino real a cholula', '20 de noviembre',
       'momoxpan ii', 'concepcion la cruz', 'concepcion guadalupe',
       'residencial del encanto', 'francisco i madero', 'villa la noria',
       'villas zavaleta primera seccion', 'santa gema',
       'villa de zavaleta', 'altavista', 'la concepcion',
       'las fuentes 

In [23]:
sin_codigo_postal_2.loc[:, 'codigo_postal'] = sin_codigo_postal_2.apply(lambda row: obtener_codigo_postal(row['Colonia_Estandarizada'], row['Municipio'], row['Estado'], df_codigos_postales), axis=1)
# Verificar si hay registros sin código postal asignado
sin_codigo_postal_3 = sin_codigo_postal_2[sin_codigo_postal_2['codigo_postal'].isna()]
limpios3 = sin_codigo_postal_2[sin_codigo_postal_2['codigo_postal'].notna()]

print(sin_codigo_postal_3[['Colonia','Municipio','Estado','codigo_postal']])
print(sin_codigo_postal_3.shape)

                          Colonia Municipio  Estado  codigo_postal
0                   puebla centro    puebla  puebla            NaN
7                   puebla centro    puebla  puebla            NaN
115                 puebla centro    puebla  puebla            NaN
130                 puebla centro    puebla  puebla            NaN
161   residencial villa encantada    puebla  puebla            NaN
...                           ...       ...     ...            ...
3464             centro historico    puebla  puebla            NaN
3465             centro historico    puebla  puebla            NaN
3466             centro historico    puebla  puebla            NaN
3470             centro historico    puebla  puebla            NaN
3475             centro historico    puebla  puebla            NaN

[333 rows x 4 columns]
(333, 32)


In [29]:
df_correctos = pd.concat([Limpios, Limpios2, limpios3])
print(df_correctos.shape)
print(df_correctos.info())

(3143, 32)
<class 'pandas.core.frame.DataFrame'>
Index: 3143 entries, 1 to 2199
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3143 non-null   int64  
 1   categoria                  2930 non-null   object 
 2   precio                     3143 non-null   float64
 3   precio_mxn                 3143 non-null   float64
 4   precio_usd                 3143 non-null   int64  
 5   fecha_conversion           3143 non-null   object 
 6   propiedad                  3143 non-null   object 
 7   metros_total               3143 non-null   int64  
 8   metros_construido          3143 non-null   int64  
 9   precio_m2_terreno          3143 non-null   float64
 10  precio_m2_construido       3143 non-null   float64
 11  tiempo_de_publicacion      3143 non-null   object 
 12  meses_transcurridos        3143 non-null   int64  
 13  meses_transcurridos_fecha  3143 non-null  

### Creación de diccionario

In [36]:
df_correctos.to_csv('colonias_estandarizadas.csv', index=False, header=True, mode='w', encoding='utf-8')

In [37]:
diccionario = pd.read_csv('colonias_estandarizadas.csv')
diccionario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3143 non-null   int64  
 1   categoria                  2930 non-null   object 
 2   precio                     3143 non-null   float64
 3   precio_mxn                 3143 non-null   float64
 4   precio_usd                 3143 non-null   int64  
 5   fecha_conversion           3143 non-null   object 
 6   propiedad                  3143 non-null   object 
 7   metros_total               3143 non-null   int64  
 8   metros_construido          3143 non-null   int64  
 9   precio_m2_terreno          3143 non-null   float64
 10  precio_m2_construido       3143 non-null   float64
 11  tiempo_de_publicacion      3143 non-null   object 
 12  meses_transcurridos        3143 non-null   int64  
 13  meses_transcurridos_fecha  3143 non-null   objec

In [42]:
# Detectar colonias estandarizadas que no están en el diccionario
colonias_no_en_diccionario = df_correctos[df_correctos['Colonia_Estandarizada'].apply(lambda x: x not in diccionario)].copy()
# Crear nuevas entradas
nuevas_entradas = pd.Series(index=colonias_no_en_diccionario['Colonia_Estandarizada'].unique(), data=colonias_no_en_diccionario['Colonia_Estandarizada'].unique())

# Actualizar el diccionario con las nuevas colonias
diccionario.update(nuevas_entradas)
diccionario.head()

,id,categoria,precio,precio_mxn,precio_usd,fecha_conversion,propiedad,metros_total,metros_construido,precio_m2_terreno,...,tipo_lugar,Colonia,Municipio,Estado,CP,ubicacion,url,descripcion,Colonia_Estandarizada,codigo_postal
0,2,E1,280000.0,280000.0,15575,2024-10-02,Venta de Departamento en Puebla de zaragoza,70,70,4000.000000,...,NaN,las hadas mundial 86,puebla,puebla,0,"Las Hadas Mundial 86, Puebla, Puebla",https://www.lamudi.com.mx/detalle/41032-73-b1c...,en iserc nuestro compromiso es que compren una...,las hadas mundial 86,72070.0
1,3,E1,315000.0,315000.0,17522,2024-10-02,"VENTA DE CASA EN PLAZAS AMALUCAN, HEROÍCA PUEB...",151,210,2086.092715,...,NaN,plazas de amalucan,puebla,puebla,0,"Plazas de Amalucan, Puebla, Puebla",https://www.lamudi.com.mx/detalle/41032-73-593...,venta de hermosa casa no se aceptan creditos ...,plazas amalucan,72310.0
2,4,E1,323656.0,323656.0,18003,2024-10-02,HERMOSA CASA EN LOS HEROES DE PUEBLA !!!,70,90,4623.657143,...,NaN,heroes de puebla,puebla,puebla,0,"Héroes de Puebla, Puebla, Puebla",https://www.lamudi.com.mx/detalle/41032-73-308...,excelente casa con la mejor ubicacion a 10 mi...,heroes de puebla,72520.0
3,5,E1,324900.0,324900.0,18073,2024-10-02,"GRAN OPORTUNIDAD, Hermosa Casa en Plazas Amalu...",150,210,2166.000000,...,NaN,plazas de amalucan,puebla,puebla,0,"Plazas de Amalucan, Puebla, Puebla",https://www.lamudi.com.mx/detalle/41032-73-95e...,recuperacion hipotecariaexcelente oportunidad ...,plazas amalucan,72310.0
4,6,E1,325000.0,325000.0,18078,2024-10-02,CASA EN VENTA ARCO TORAL SAN JOSE MAYORAZGO PU...,85,100,3823.529412,...,NaN,san jose mayorazgo,puebla,puebla,0,"San José Mayorazgo, Puebla, Puebla",https://www.lamudi.com.mx/detalle/41032-73-aed...,casa en venta la oportunidad de adquirir un in...,san jose mayorazgo,72450.0


In [31]:
dataos = df_codigos_postales[df_codigos_postales['d_asenta'].str.contains('atlixco', case=False)]
dataos['d_asenta'].value_counts()

d_asenta
villas de atlixco                         1
corredor empresarial boulevard atlixco    1
atlixco centro                            1
real conveima atlixco                     1
atlixco 90                                1
san pedro atlixco                         1
Name: count, dtype: int64

In [27]:
df_codigos_postales['d_asenta'].unique()

array(['centro', 'san francisco', 'santa maria la rivera', ...,
       'xochiapa', 'ixtacxochitla', 'tequitlale'], dtype=object)

1213 -> 1124 -> 1063

In [55]:
centro_puebla = dataos[(dataos['d_asenta'] == 'centro') & (dataos['D_mnpio'].str.contains('Puebla', case=False))]
centro_puebla

,d_codigo,d_asenta,d_tipo_asenta,D_mnpio,estado


In [50]:
print(df[['tipo_lugar','Colonia','Colonia_Estandarizada','Municipio','Estado','ubicacion']].head(35))

   tipo_lugar                         Colonia           Colonia_Estandarizada  \
0        None                   puebla centro                   puebla centro   
1        None            las hadas mundial 86            las hadas mundial 86   
2        None                 plazas amalucan                 plazas amalucan   
3        None                heroes de puebla         los heroes de puebla ii   
4        None                 plazas amalucan                 plazas amalucan   
5        None              san jose mayorazgo              san jose mayorazgo   
6        None           ex-hacienda mayorazgo           ex-hacienda mayorazgo   
7        None                   puebla centro                   puebla centro   
8        None                       la calera           plan de ayala caleras   
9        None                       el carmen                nueva del carmen   
10       None                     santa maria           santa maria la rivera   
11       None               

In [10]:
from rapidfuzz import process

#Lista de colonias estandarizadas
colonias_estandar = ['centro historico','colonia 2']

#Aplicar fuzzy matching para corregir por similitud((
df['Colonia'] = df['Colonia'].apply(lambda x: process.extractOne(x, colonias_estandar)[0])
df['Colonia'].unique()

In [54]:
print(df[['tipo_lugar','Colonia','Colonia_Estandarizada','Municipio','Estado','ubicacion']].head(35))

   tipo_lugar                         Colonia           Colonia_Estandarizada  \
0        None                   puebla centro                   puebla centro   
1        None            las hadas mundial 86            las hadas mundial 86   
2        None              plazas de amalucan              plazas de amalucan   
3        None                heroes de puebla                heroes de puebla   
4        None              plazas de amalucan              plazas de amalucan   
5        None              san jose mayorazgo              san jose mayorazgo   
6        None           ex-hacienda mayorazgo           ex-hacienda mayorazgo   
7        None                   puebla centro                   puebla centro   
8        None                       la calera           plan de ayala caleras   
9        None                       el carmen                nueva del carmen   
10       None                     santa maria                     santa maria   
11       None               